# press shift enter to execute the cells until you reach the place to write SQL

In [2]:
import sqlite3
import pandas as pd
pd.options.display.max_rows = 5000

def q(query):
    return pd.read_sql(query,connect)
from google.colab import drive
drive.mount('/content/drive')
connect = sqlite3.connect('sql_course.db')
print("connected successfully")

Mounted at /content/drive
connected successfully


In [4]:
customer = pd.read_csv('https://raw.githubusercontent.com/Yasser-AlaaEldin/sql-course-material/main/foodies%20-%20customer.csv') # load to DataFrame
marketing = pd.read_csv('https://raw.githubusercontent.com/Yasser-AlaaEldin/sql-course-material/main/foodies%20-%20marketing.csv') # load to DataFrame
complaints = pd.read_csv('https://raw.githubusercontent.com/Yasser-AlaaEldin/sql-course-material/main/foodies%20-%20complaints.csv') # load to DataFrame
restaurants = pd.read_csv('https://raw.githubusercontent.com/Yasser-AlaaEldin/sql-course-material/main/foodies%20-%20restaurants.csv') # load to DataFrame
orders = pd.read_csv('https://raw.githubusercontent.com/Yasser-AlaaEldin/sql-course-material/main/foodies%20-%20orders.csv') # load to DataFrame

connect.execute("drop table if exists customer")
connect.commit()
connect.cursor().close()
connect.execute("""create table if not exists customer(customer_id integer,customer_name varchar,city varchar,phone integer,subscribtion_date date,gender varchar,age integer)""")
connect.commit()
connect.cursor().close()
customer.to_sql('customer', connect, if_exists='append', index = False) # write to sqlite table

q("select * from customer")

connect.execute("drop table if exists marketing")
connect.commit()
connect.cursor().close()
connect.execute("""create table if not exists marketing(day date,number_comms integer,cost integer,number_of_signup_users integer,channel varchar)""")
connect.commit()
connect.cursor().close()
marketing.to_sql('marketing', connect, if_exists='append', index = False) # write to sqlite table

q("select * from marketing")

connect.execute("drop table if exists complaints")
connect.commit()
connect.cursor().close()
connect.execute("""create table if not exists complaints(complain_id integer,complain_reason varchar,order_id integer)""")
connect.commit()
connect.cursor().close()
complaints.to_sql('complaints', connect, if_exists='append', index = False) # write to sqlite table

q("select * from complaints")

connect.execute("drop table if exists restaurants")
connect.commit()
connect.cursor().close()
connect.execute("""create table if not exists restaurants(restaurant_id integer,restaurant_name varchar,restaurant_category varchar,subscribtion_start_date date,subscribtion_end_date date,subscribtion_status varchar)""")
connect.commit()
connect.cursor().close()
restaurants.to_sql('restaurants', connect, if_exists='append', index = False) # write to sqlite table

q("select * from restaurants")

connect.execute("drop table if exists orders")
connect.commit()
connect.cursor().close()
connect.execute("""create table if not exists orders(customer_id integer,order_id integer,paid_amount integer,payment_type varchar,discount_amount integer,promo_code varchar,restaurant_id integer,ordered_time datetime,arrival_time datetime,order_date date
)""")
connect.commit()
connect.cursor().close()
orders.to_sql('orders', connect, if_exists='append', index = False) # write to sqlite table



602

<BR>**table names: orders,restaurants,marketing,complaints,customer**

In [5]:
q(""" select *  , count(*) from orders limit 3


""")

,customer_id,order_id,paid_amount,payment_type,discount_amount,promo_code,restaurant_id,ordered_time,arrival_time,order_date,count(*)
0,63,1,591,visa,10,pizza40,7,2023-01-23 14:48,2023-01-23 15:18,2023-01-23,602


In [6]:
q(""" select * , count(*) from restaurants limit 3

""")

,restaurant_id,restaurant_name,restaurant_category,subscribtion_start_date,subscribtion_end_date,subscribtion_status,count(*)
0,1,best crep,fast food,2022-01-02,None,live,35


In [7]:
q(""" select * , count(*) from marketing limit 3

""")

,day,number_comms,cost,number_of_signup_users,channel,count(*)
0,2023-02-11,1674,33.5,498,sms,60


In [8]:
q(""" select * , count(*) from complaints limit 3

""")

,complain_id,complain_reason,order_id,count(*)
0,1,delivery delay,354,99


In [9]:
q(""" select * , count(*) from customer limit 3

""")

,customer_id,customer_name,city,phone,subscribtion_date,gender,age,count(*)
0,1,SMITH,cairo,20190783845,2022-01-02,male,17,119


## Demographics analysis
- How is the base distributed among males and females ?
  - (how many users in each category male/female and the % of each out of the total user base)
- which age segment of people are attracted to our platform ?
  - (make a new column using case when and segment customers into 3 segments children,youth,old based on cutoffs of your choice and then get the count of each and the % of each out of total base)

- how is the distribution of the regions 
  - what is the top city we have users in
  
- Hint: there might be some outliers in the age or missing data .. propose a way to deal with it (you can use case when)

In [10]:
# First cleaning and removing the null values

In [11]:
q("""
select count(customer_id) from customer

""")
# so there is no duplicated

,count(customer_id)
0,119


In [47]:
q("""
delete  from customer where city is null

""")

TypeError: ignored

In [48]:
q(""" select * from customer where city is null
""")



,customer_id,customer_name,city,phone,subscribtion_date,gender,age


In [92]:
q("""

delete from customer where age > 90

""")

TypeError: ignored

In [94]:
q("""
select * from customer where age > 90


""")

,customer_id,customer_name,city,phone,subscribtion_date,gender,age


In [95]:
# males and females percentage as customers
q(""" select count(gender) as count,(count(gender)*100.0 / (select count(*) from customer)) as percentage, gender from customer
group by gender
""")

,count,percentage,gender
0,30,26.785714,female
1,82,73.214286,male


In [96]:
q(""" 
select count(*), ( count(*)* 100.0/ (select count(*) from customer) ) as Perc ,
case when age between 1 and 12 then 'Child'
when age between 13 and 18 then 'Teen'
when age between 19 and 40 then ' Youth (+18)'
when age > 40 then 'Old (+40)'
end as Age_group
from customer
group by Age_group

""")

,count(*),Perc,Age_group
0,55,49.107143,Youth (+18)
1,51,45.535714,Old (+40)
2,6,5.357143,Teen


In [97]:
q(""" 
select city  , count( city) as city_count from customer 

group by city 

order by count(city) desc



""")

,city,city_count
0,cairo,56
1,alex,40
2,tanta,8
3,mansoura,8


In [98]:
q("""
select city  , count( city) as city_count from customer 

group by city 

order by count(city) desc
limit 1

""")

,city,city_count
0,cairo,56


## ordering performance analysis
- Can you analyze the main kpis for the online food business to check how successful is the business right now?
- Calculate the overall OPU
  - OPU: average orders per user ?
  - OPU = total number of orders / unique count of users
- Calculate the overall ARPU
  - ARPU: average Revenue per user ?
  - ARPU = total Revenue of orders / unique count of users

- compare between the behaviour of cash/visa users to see which cohort type is generating more value to us
  - calculate the ARPU and the OPU for each group
- build a report to show the daily revenues incoming to the platform, and the number of orders everyday
  - for each day print out the summation of revenues and the number of orders
- how good is our delivery service ?
  - get the overall avg delivery time (selft study how to use strftime() function in sqlite)

- compare between the behaviour of males and females and the different age segments 
  - get the ARPU andthe OPU for each bucket
  - get the favourite food category for each bucket (highest food category that has orders)

In [99]:
q("""
select * , count(*) from orders limit 3

""")

,customer_id,order_id,paid_amount,payment_type,discount_amount,promo_code,restaurant_id,ordered_time,arrival_time,order_date,count(*)
0,63,1,591,visa,10,pizza40,7,2023-01-23 14:48,2023-01-23 15:18,2023-01-23,602


In [100]:
# first check for duplicates and null values

In [101]:
q("""
select count(*), count(order_id) from orders
group by order_id
having count(*) > 1
 -- for checking duplicates
""")

,count(*),count(order_id)


In [102]:
q("""
select * from orders where order_id is null
-- for checking nulls
""")

,customer_id,order_id,paid_amount,payment_type,discount_amount,promo_code,restaurant_id,ordered_time,arrival_time,order_date


In [103]:
q("""
-- calculating Average Order Per User
select count(distinct order_id)*1.00/count(distinct customer_id) from orders

""")

,count(distinct order_id)*1.00/count(distinct customer_id)
0,5.960396


In [104]:
q("""
-- validating out calculation
select max(order_id),max(customer_id) , min(order_id),min(customer_id) from orders
-- therefor it is valid as 601/101 = 5.96
""")

,max(order_id),max(customer_id),min(order_id),min(customer_id)
0,602,100,1,0


In [105]:
q("""
-- Calculating ARPU
select sum(paid_amount)*1.0/count(distinct customer_id) as ARPU from orders

""")

,ARPU
0,3000.732673


In [106]:
q("""
-- getting the total revenue and validating
select sum(paid_amount) , count( distinct customer_id) from orders

""")

,sum(paid_amount),count( distinct customer_id)
0,303074,101


In [107]:
q("""
-- Payment method analysis
-- Calculating OPU , ARPU for each payment method
select payment_type , count(*) , sum(paid_amount) ,round ((count(*)*100.00 / (select count(*) from orders)),2) as Count_Perc ,
round((sum(paid_amount)*100.0 / (select sum(paid_amount) from orders)),2) as Paid_Perc , 
round(count(order_id)*1.0/count(distinct customer_id),2) as OPU  ,
sum(paid_amount)/count(distinct customer_id)*1.0 as APRU

from orders 
group by payment_type


""")

,payment_type,count(*),sum(paid_amount),Count_Perc,Paid_Perc,OPU,APRU
0,cash,456,228187,75.75,75.29,4.56,2281.0
1,visa,146,74887,24.25,24.71,1.85,947.0


In [108]:
q("""
-- now we know that the highest payments were done in cash and not visa
select paid_amount , payment_type from orders 

order by paid_amount desc
limit 5

""")

,paid_amount,payment_type
0,998,cash
1,994,cash
2,993,cash
3,991,cash
4,990,cash


In [109]:
q("""
-- count of orders and sum of revenue daily
select order_date, count(order_id) as 'num_orders/day' , sum(paid_amount)  as 'revenue/day' from orders
group by order_date


""")

,order_date,num_orders/day,revenue/day
0,2023-01-23,11,6123
1,2023-01-24,14,5500
2,2023-01-25,5,2311
3,2023-01-26,13,5965
4,2023-01-27,14,5854
5,2023-01-28,7,3039
6,2023-01-29,10,4611
7,2023-01-30,14,7564
8,2023-01-31,16,9278
9,2023-02-01,23,12615


In [110]:
q("""
select * from orders limit 5

""")

,customer_id,order_id,paid_amount,payment_type,discount_amount,promo_code,restaurant_id,ordered_time,arrival_time,order_date
0,63,1,591,visa,10,pizza40,7,2023-01-23 14:48,2023-01-23 15:18,2023-01-23
1,94,2,867,cash,4,pizza40,17,2023-02-10 16:52,2023-02-10 17:13,2023-02-10
2,37,3,442,visa,5,talabat hero,15,2023-02-09 17:16,2023-02-09 18:14,2023-02-09
3,0,4,557,cash,8,pizza40,2,2023-02-05 18:04,2023-02-05 18:10,2023-02-05
4,92,5,372,visa,6,pizza40,2,2023-02-03 10:28,2023-02-03 11:19,2023-02-03


In [111]:
q("""
SELECT strftime('%Y-%m-%d-%H-%M', ordered_time) from orders
limit 5


""")

,"strftime('%Y-%m-%d-%H-%M', ordered_time)"
0,2023-01-23-14-48
1,2023-02-10-16-52
2,2023-02-09-17-16
3,2023-02-05-18-04
4,2023-02-03-10-28


In [112]:
q("""
--Max time taken is 59 minuetes which is Good to our standards

SELECT (strftime('%s', arrival_time) - strftime('%s', ordered_time)) /60 AS delivery_time_minutes

FROM orders
order by delivery_time_minutes desc
limit 3

""")

,delivery_time_minutes
0,59
1,59
2,59


In [113]:
q("""
-- Getting the average delivery time , almost 30 minutes
select round(avg((strftime ('%s', arrival_time) - strftime('%s',ordered_time)) / 60),2) || ' Minutes' as AVG_DELIV_TIME from orders

""")

,AVG_DELIV_TIME
0,29.21 Minutes


compare between the behaviour of males and females and the different age segments

get the ARPU andthe OPU for each bucket
get the favourite food category for each bucket (highest food category that has orders)

In [114]:
q("""
select * from orders limit 1

""")

,customer_id,order_id,paid_amount,payment_type,discount_amount,promo_code,restaurant_id,ordered_time,arrival_time,order_date
0,63,1,591,visa,10,pizza40,7,2023-01-23 14:48,2023-01-23 15:18,2023-01-23


In [115]:
q("""
select * from customer limit 1

""")

,customer_id,customer_name,city,phone,subscribtion_date,gender,age
0,1,SMITH,cairo,20190783845,2022-01-02,male,17


In [116]:
q("""
--Total paid vs gender
-- we shall use inner join 
select customer.gender , 
-- commenting the Age_group
/*case when customer.age between 1 and 12 then 'Child'
when customer.age between 13 and 18 then 'Teen'
when customer.age between 19 and 40 then ' Youth (+18)'
when customer.age > 40 then 'Old (+40)'
end as Age_group, */
sum(orders.paid_amount) || ' $' as TOTAL_PAID ,/* orders.payment_type ,*/ count(orders.payment_type) as GD_COUNT
,round((sum(orders.paid_amount)*100.0 / (select sum(paid_amount) from orders)) , 2)as PAID_GD_PERC
from customer
inner join orders on customer.customer_id=orders.customer_id
group by  customer.gender 


""")

,gender,TOTAL_PAID,GD_COUNT,PAID_GD_PERC
0,female,77612 $,152,25.61
1,male,200047 $,403,66.01


In [117]:
q("""

-- payment type vs gender
select customer.gender , 
-- commenting the Age_group
/*case when customer.age between 1 and 12 then 'Child'
when customer.age between 13 and 18 then 'Teen'
when customer.age between 19 and 40 then ' Youth (+18)'
when customer.age > 40 then 'Old (+40)'
end as Age_group, 
sum(orders.paid_amount) as TOTAL_PAID ,*/ orders.payment_type , count(orders.payment_type) as COUNT ,
round((count(orders.payment_type)*100.0/ (select count(*) from orders )) ,2)|| '%' as PERC
from customer
inner join orders on customer.customer_id=orders.customer_id
group by  customer.gender ,orders.payment_type 
order by COUNT desc


""")

,gender,payment_type,COUNT,PERC
0,male,cash,303,50.33%
1,female,cash,117,19.44%
2,male,visa,100,16.61%
3,female,visa,35,5.81%


In [118]:
q("""

-- payment type vs AGE
select
-- commenting the Age_group
case when customer.age between 1 and 12 then 'Child'
when customer.age between 13 and 18 then 'Teen'
when customer.age between 19 and 40 then ' Youth (+18)'
when customer.age > 40 then 'Old (+40)'
end as Age_group, 
orders.payment_type , count(orders.payment_type) as COUNT ,
round((count(orders.payment_type)*100.0/ (select count(*) from orders )) ,2) || '%'as PERC
from customer
inner join orders on customer.customer_id=orders.customer_id
group by  Age_group , orders.payment_type
order by COUNT desc


""")

,Age_group,payment_type,COUNT,PERC
0,Youth (+18),cash,234,38.87%
1,Old (+40),cash,164,27.24%
2,Youth (+18),visa,70,11.63%
3,Old (+40),visa,61,10.13%
4,Teen,cash,22,3.65%
5,Teen,visa,4,0.66%


get the ARPU andthe OPU for each bucket get the favourite food category for each bucket (highest food category that has orders)
table names: orders,restaurants,marketing,complaints,customer

In [119]:
q("""
select * from customer limit 1
""")

,customer_id,customer_name,city,phone,subscribtion_date,gender,age
0,1,SMITH,cairo,20190783845,2022-01-02,male,17


In [120]:
q("""
select * from orders limit 1
""")

,customer_id,order_id,paid_amount,payment_type,discount_amount,promo_code,restaurant_id,ordered_time,arrival_time,order_date
0,63,1,591,visa,10,pizza40,7,2023-01-23 14:48,2023-01-23 15:18,2023-01-23


In [121]:
q("""
select * from restaurants 
limit 1
""")

,restaurant_id,restaurant_name,restaurant_category,subscribtion_start_date,subscribtion_end_date,subscribtion_status
0,1,best crep,fast food,2022-01-02,None,live


In [122]:
q(""" -- Here we find the most loved food
select restaurants.restaurant_category ,
/* case when customer.age between 1 and 12 then 'Child'
when customer.age between 13 and 18 then 'Teen'
when customer.age between 19 and 40 then ' Youth (+18)'
when customer.age > 40 then 'Old (+40)'
end as Age_group , */count(orders.order_id) , 
round((count(orders.order_id)*100.0 /(select count(*) from orders)),2) || '%' as PERC
from restaurants
inner join orders on restaurants.restaurant_id=orders.restaurant_id
inner join customer on orders.customer_id=customer.customer_id
group by restaurant_category
order by count(orders.order_id)  desc
""")

,restaurant_category,count(orders.order_id),PERC
0,fast food,462,76.74%
1,drinks,50,8.31%
2,home made,32,5.32%
3,dessert,11,1.83%


In [123]:
q(""" -- what each age category likes ( EVERYONE LOVES FAST FOOD )
select restaurants.restaurant_category ,
 case when customer.age between 1 and 12 then 'Child'
when customer.age between 13 and 18 then 'Teen'
when customer.age between 19 and 40 then ' Youth (+18)'
when customer.age > 40 then 'Old (+40)'
end as Age_group , count(orders.order_id) from restaurants
inner join orders on restaurants.restaurant_id=orders.restaurant_id
inner join customer on orders.customer_id=customer.customer_id
group by Age_group 
order by count(orders.order_id) desc
""")

,restaurant_category,Age_group,count(orders.order_id)
0,fast food,Youth (+18),304
1,fast food,Old (+40),225
2,fast food,Teen,26


## Restaurants analysis
- please do an analysis to show case how good our restaurants are serving our customers in the App
- get number of orders for each restaurant and the % of each out of total orders
- get the top 3 restaurants according to orders (using window functions)
- get the revenue for each restaurant and the % of each out of total revenue
- get the top 3 restaurants according to revenues (using window functions)
- how many complain for each restaurant
- please get the restaurant that has the highest number of complain

In [124]:
q("""
select restaurants.restaurant_name as Name, count(orders.order_id) as number_of_orders , round((count(orders.order_id)*100.0/ (select count(order_id) from orders)),2) As 'Perc%'
from orders
inner join restaurants on orders.restaurant_id=restaurants.restaurant_id
group by restaurants.restaurant_name
order by count(orders.order_id )desc
""")

,Name,number_of_orders,Perc%
0,mcdonalds,200,33.22
1,papa jones,85,14.12
2,shawermer,53,8.80
3,el eman,45,7.48
4,belal,18,2.99
5,zezo,16,2.66
6,shawerma el reem,14,2.33
7,ma2moon drinks,14,2.33
8,drink cola,13,2.16
9,passtawessy,12,1.99


In [125]:
q("""
select distinct restaurants.restaurant_name as Name, count(orders.order_id) over(partition by  restaurants.restaurant_id) as number_of_orders 
from orders
inner join restaurants on orders.restaurant_id=restaurants.restaurant_id
 order by number_of_orders desc
limit  3

""")

,Name,number_of_orders
0,mcdonalds,200
1,papa jones,85
2,shawermer,53


get the revenue for each restaurant and the % of each out of total revenue

In [126]:
q("""
select * from restaurants limit 1

""")

,restaurant_id,restaurant_name,restaurant_category,subscribtion_start_date,subscribtion_end_date,subscribtion_status
0,1,best crep,fast food,2022-01-02,None,live


In [127]:
q("""
select r.restaurant_name , sum(o.paid_amount) as revenue ,
round ((sum(o.paid_amount)* 100.0) /(select sum(paid_amount) from orders),2) || '%' as perc
 from orders as o
inner join restaurants as r on o.restaurant_id=r.restaurant_id
group by r.restaurant_name 
order by revenue desc
""")

,restaurant_name,revenue,perc
0,mcdonalds,103853,34.27%
1,papa jones,44578,14.71%
2,shawermer,30193,9.96%
3,el eman,18421,6.08%
4,zezo,10189,3.36%
5,belal,7868,2.6%
6,drink cola,7070,2.33%
7,ma2moon drinks,6873,2.27%
8,super mama,6247,2.06%
9,passtawessy,6206,2.05%


- get the top 3 restaurants according to revenues (using window functions)

In [128]:
q("""
select distinct r.restaurant_name , sum(o.paid_amount) over(partition by r.restaurant_name) as "revenue" from orders as o
inner join restaurants as r on o.restaurant_id=r.restaurant_id
order by "revenue" desc
limit 3
""")

,restaurant_name,revenue
0,mcdonalds,103853
1,papa jones,44578
2,shawermer,30193


how many complain for each restaurant
please get the restaurant that has the highest number of complain

In [129]:
q("""
select r.restaurant_name as rest_name , count(c.complain_id) as "complains_no" from complaints as c 
inner join orders as o on c.order_id = o.order_id
inner join restaurants as r on o.restaurant_id = r.restaurant_id
group by rest_name
order by complains_no desc 
-- limit 1
""")

,rest_name,complains_no
0,mcdonalds,37
1,papa jones,17
2,shawermer,5
3,waffilisious,4
4,super mama,4
5,el eman,4
6,panda house,3
7,abo mazen,3
8,zezo,2
9,passtawessy,2


## Complaints analysis
- please find out the number one complain reason the customers are complaining about

- we want to see the effect of the complains on the customers 
  - please make two groups: Group A: users that have at least one complaint 
  - group B: users that have no complaints
  - and then calculate the ARPU and the OPU for each group

In [130]:
q("""
select complain_reason , count(complain_id) as count from complaints
group by complain_reason
order by count desc
-- limit 1
""")

,complain_reason,count
0,delivery delay,60
1,food was cold,20
2,unprofessional delivery man,7
3,food quality is low,5
4,bad packaging,5
5,missing items,2


please make two groups: Group A: users that have at least one complaint
group B: users that have no complaints
and then calculate the ARPU and the OPU for each group

In [131]:
q("""
select  count( distinct customer_id) from orders

""")

,count( distinct customer_id)
0,101


In [132]:
q("""
select

case when c.complain_id is null then ' No complains '
when c.complain_id is not null then '  compalins '
end as status  , count ( distinct o.customer_id) as count
from orders as o 
left join complaints as c on  o.order_id = c.order_id 
group by status
""")

,status,count
0,compalins,62
1,No complains,101


# Marketing performance
- calculate the CAC and the conversion rate for each channel

In [140]:
q("""
select channel, avg(cost)  , count(channel) ,  (number_of_signup_users *100.0/ number_comms) as 'Conversion Rate' from marketing
group by channel
""")

,channel,avg(cost),count(channel),Conversion Rate
0,email,32.450000,4,6.533524
1,facebook ads,31.013889,36,5.861785
2,google ads,45.683333,6,7.153429
3,sms,55.166667,6,29.749104
4,whatsapp,31.012500,8,7.529412
